In [1]:
import mne
import numpy as np

import os

import matplotlib.pyplot as plt

from sklearn.utils import resample

In [2]:
directory = './eeg-during-mental-arithmetic-tasks-1.0.0/'

rest_filepaths = []
task_filepaths = []

for filename in os.listdir(directory):
    filepath = os.path.join(directory, filename)
    if filename.endswith('.edf'):
        label = filename.split('_')[-1].split('.')[0]

        if label == '1':
            rest_filepaths.append(filepath)
        else:
            task_filepaths.append(filepath)

In [3]:
len(rest_filepaths), len(task_filepaths)

(36, 36)

In [4]:
average_epochs = 81


def read_data(filepath):
    data = mne.io.read_raw_edf(filepath, preload=True)
    data.set_eeg_reference()
    data.filter(l_freq=0.5, h_freq=45)
    tmin, tmax = 0, 61.9
    data.crop(tmin=tmin, tmax=tmax)
    # epochs = mne.make_fixed_length_epochs(data, duration=2, overlap=1)
    # array = epochs.get_data()

    # if array.shape[0]>121:
    #     array = resample(array, replace=False, n_samples=average_epochs, random_state=42)
    # else:
    #     # Oversample 'task' epochs to the average value
    #     array = resample(array, replace=True, n_samples=average_epochs, random_state=42)
    # return array
    return data.get_data()

In [5]:
# average_epochs = 81


# def read_data(filepath):
#     data = mne.io.read_raw_edf(filepath, preload=True)
#     data.set_eeg_reference()
#     data.filter(l_freq=0.5, h_freq=45)

#     data1, times1 = data[:, :int(60 * data.info['sfreq'])]

#     return data1

In [6]:
%%capture
rest_epochs_array = [read_data(filepath) for filepath in rest_filepaths]
task_epochs_array = [read_data(filepath) for filepath in task_filepaths]

In [7]:
rest_epochs_array[0].shape, task_epochs_array[0].shape

((21, 30951), (21, 30951))

In [8]:
len(rest_epochs_array)

36

In [9]:
rest_epochs_array = np.array(rest_epochs_array)
task_epochs_array = np.array(task_epochs_array)
rest_epochs_array.shape

(36, 21, 30951)

In [10]:
# Labels:
rest_label = [0 for _ in rest_epochs_array]
task_label = [1 for _ in task_epochs_array]

In [11]:
len(task_label)

36

In [12]:
all_epochs = np.concatenate((rest_epochs_array, task_epochs_array))
all_labels = rest_label + task_label

In [13]:
len(all_labels)

72

In [14]:
perm = np.random.permutation(72)

# Shuffle both arrays using the same permutation along the first axis
shuffled_epochs = []
shuffled_labels = []

for index in perm:
    shuffled_epochs.append(all_epochs[index])
    shuffled_labels.append(all_labels[index])

all_epochs = shuffled_epochs
all_labels = shuffled_labels

In [15]:
len(all_epochs), len(all_epochs[0]), len(all_epochs[0][0])

(72, 21, 30951)

In [16]:
# group_list = [[i]*len(j) for i, j in enumerate(all_epochs)]

In [17]:
# len(group_list), len(all_epochs), len(all_labels)

In [18]:
# data_array = np.vstack(all_epochs)
# label_array = np.hstack(all_labels)
# group_array = np.hstack(group_list)
data_array = np.array(all_epochs)
label_array = np.array(all_labels)

In [19]:
print(data_array.shape, label_array.shape)

(72, 21, 30951) (72,)


In [20]:
# Normalize
import numpy as np
eeg_data= data_array


# Assuming your EEG data is stored in a variable called `eeg_data`
# Shape of eeg_data: (72, 21, 30951)

# Initialize an array to store the normalized data
normalized_data = np.zeros_like(eeg_data)

means = np.mean(eeg_data, axis=(0, 2))
stds = np.std(eeg_data, axis=(0, 2))

# Normalize each channel for all candidates
for candidate in range(eeg_data.shape[0]):
    for channel in range(eeg_data.shape[1]):
        normalized_data[candidate, channel, :] = (eeg_data[candidate, channel, :] - means[channel]) / stds[channel]

# normalized_eeg_data now contains the z-score normalized EEG data


In [21]:
normalized_data.shape

(72, 21, 30951)

In [22]:
from tensorflow.keras.utils import to_categorical

In [23]:
label_array_one_hot = to_categorical(label_array)
label_array_one_hot.shape

(72, 2)

In [24]:
from sklearn.model_selection import train_test_split

train_features, test_features, train_labels, test_labels = train_test_split(normalized_data, label_array_one_hot, test_size=0.2, random_state=42)
print("Shape of train_features:", train_features.shape)
print("Shape of train_labels:", train_labels.shape)
print("Shape of test_features:", test_features.shape)
print("Shape of test_labels:", test_labels.shape)

Shape of train_features: (57, 21, 30951)
Shape of train_labels: (57, 2)
Shape of test_features: (15, 21, 30951)
Shape of test_labels: (15, 2)


# WAY TO TRAINING


In [25]:
ones = 0
for item in test_labels:
    if item[0] ==0:
        ones = 1+ones
    
print(ones)

8


### Model 1


In [33]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Permute, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import SeparableConv2D, DepthwiseConv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import SpatialDropout2D
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.layers import Input, Flatten
from tensorflow.keras.constraints import max_norm
from tensorflow.keras import backend as K

from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.losses import BinaryCrossentropy




# Example function for EEGNet model


def EEGNet(nb_classes, Chans=64, Samples=128,
           dropoutRate=0.5, kernLength=64, F1=64,
           D=2, F2=16, norm_rate=0.25, dropoutType='Dropout'):

    input1   = Input(shape = (Chans, Samples, 1))

    ##################################################################
    block1       = Conv2D(F1, (1, kernLength), padding = 'same',
                                   input_shape = (Chans, Samples, 1),
                                   use_bias = False)(input1)
    block1       = BatchNormalization()(block1)
    block1       = DepthwiseConv2D((Chans, 1), use_bias = False, 
                                   depth_multiplier = D,
                                   depthwise_constraint = max_norm(1.))(block1)
    block1       = BatchNormalization()(block1)
    block1       = Activation('elu')(block1)
    block1       = AveragePooling2D((1, 4))(block1)
    block1       = Dropout(dropoutRate)(block1)
    
    block2       = SeparableConv2D(F2, (1, 16),
                                   use_bias = False, padding = 'same')(block1)
    block2       = BatchNormalization()(block2)
    block2       = Activation('elu')(block2)
    block2       = AveragePooling2D((1, 8))(block2)
    block2       = Dropout(dropoutRate)(block2)
        
    flatten      = Flatten(name = 'flatten')(block2)
    
    dense        = Dense(nb_classes, name = 'dense', 
                         kernel_constraint = max_norm(norm_rate))(flatten)
    softmax      = Activation('softmax', name = 'softmax')(dense)
    
    return Model(inputs=input1, outputs=softmax)

model_1 = EEGNet(2, Chans=data_array.shape[1],Samples=data_array.shape[2] )
model_1.summary()

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 21, 30951, 1)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 21, 30951, 64)  │         4,096 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_12          │ (None, 21, 30951, 64)  │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ depthwise_conv2d_4              │ (None, 1, 30951, 128)  │         2,688 │
│ (DepthwiseConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_13          │ (None, 1, 30951, 128)  │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_8 (Activation)       │ (None, 1, 30951, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_8             │ (None, 1, 7737, 128)   │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 1, 7737, 128)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ separable_conv2d_4              │ (None, 1, 7737, 16)    │         4,096 │
│ (SeparableConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_14          │ (None, 1, 7737, 16)    │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_9 (Activation)       │ (None, 1, 7737, 16)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_9             │ (None, 1, 967, 16)     │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 1, 967, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 15472)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2)              │        30,946 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ softmax (Activation)            │ (None, 2)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 42,658 (166.63 KB)

 Trainable params: 42,242 (165.01 KB)

 Non-trainable params: 416 (1.62 KB)

In [34]:
# Compile model
model_1.compile(optimizer='adam', loss=BinaryCrossentropy(),
                metrics=['accuracy'])

# Train model on current fold's train and validation data

model_1.fit(train_features, train_labels, epochs=2,batch_size=1, validation_split=0.2)

# Evaluate model on test data for this fold
test_loss, test_acc = model_1.evaluate(test_features, test_labels)

Epoch 1/2
45/45 ━━━━━━━━━━━━━━━━━━━━ 34s 637ms/step - accuracy: 0.5412 - loss: 0.6638 - val_accuracy: 0.5000 - val_loss: 0.6928
Epoch 2/2
45/45 ━━━━━━━━━━━━━━━━━━━━ 22s 493ms/step - accuracy: 0.9768 - loss: 0.4021 - val_accuracy: 0.5833 - val_loss: 0.6858
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.6000 - loss: 0.6968


### Model 2


In [28]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Permute, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import SeparableConv2D, DepthwiseConv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import SpatialDropout2D
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.layers import Input, Flatten
from tensorflow.keras.constraints import max_norm
from tensorflow.keras import backend as K

from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.losses import BinaryCrossentropy



def EEGNet(nb_classes, Chans=64, Samples=128,
           dropoutRate=0.5, kernLength=128, F1=256,
           D=2, F2=128, norm_rate=0.25, dropoutType='Dropout'):
    
    if dropoutType == 'SpatialDropout2D':
        dropoutType = SpatialDropout2D
    elif dropoutType == 'Dropout':
        dropoutType = Dropout
    else:
        raise ValueError('dropoutType must be one of SpatialDropout2D '
                         'or Dropout, passed as a string.')
    
    input1   = Input(shape = (Chans, Samples, 1))

    ##################################################################
    block1       = Conv2D(F1, (1, kernLength), padding = 'same',
                                   input_shape = (Chans, Samples, 1),
                                   use_bias = False)(input1)
    block1       = BatchNormalization()(block1)
    block1       = DepthwiseConv2D((Chans, 1), use_bias = False, 
                                   depth_multiplier = D,
                                   depthwise_constraint = max_norm(1.))(block1)
    block1       = BatchNormalization()(block1)
    block1       = Activation('elu')(block1)
    block1       = AveragePooling2D((1, 4))(block1)
    block1       = SpatialDropout2D(dropoutRate)(block1)
    
    block2       = SeparableConv2D(F2, (1, 16),
                                   use_bias = False, padding = 'same')(block1)
    block2       = BatchNormalization()(block2)
    block2       = Activation('elu')(block2)
    block2       = AveragePooling2D((1, 8))(block2)
    block2       = SpatialDropout2D(dropoutRate)(block2)
        
    flatten      = Flatten(name = 'flatten')(block2)
    
    dense        = Dense(nb_classes, name = 'dense', 
                         kernel_constraint = max_norm(norm_rate))(flatten)
    softmax      = Activation('softmax', name = 'softmax')(dense)
    
    return Model(inputs=input1, outputs=softmax)

model_2 = EEGNet(2, Chans=data_array.shape[1],Samples=data_array.shape[2] )
model_2.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 21, 30951, 1)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 21, 30951, 256) │        32,768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 21, 30951, 256) │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ depthwise_conv2d_1              │ (None, 1, 30951, 512)  │        10,752 │
│ (DepthwiseConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 1, 30951, 512)  │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 1, 30951, 512)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_2             │ (None, 1, 7737, 512)   │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout2d               │ (None, 1, 7737, 512)   │             0 │
│ (SpatialDropout2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ separable_conv2d_1              │ (None, 1, 7737, 128)   │        73,728 │
│ (SeparableConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 1, 7737, 128)   │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 1, 7737, 128)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_3             │ (None, 1, 967, 128)    │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout2d_1             │ (None, 1, 967, 128)    │             0 │
│ (SpatialDropout2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 123776)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2)              │       247,554 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ softmax (Activation)            │ (None, 2)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 368,386 (1.41 MB)

 Trainable params: 366,594 (1.40 MB)

 Non-trainable params: 1,792 (7.00 KB)

In [29]:
# # Compile model
# model_2.compile(optimizer='adam', loss=BinaryCrossentropy(),
#                 metrics=['accuracy'])

# # Train model on current fold's train and validation data

# model_2.fit(train_features, train_labels, epochs=5,batch_size=1, validation_split=0.2)

# # Evaluate model on test data for this fold
# test_loss, test_acc = model_2.evaluate(test_features, test_labels)

### Model 3


In [31]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Permute, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import SeparableConv2D, DepthwiseConv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import SpatialDropout2D
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.layers import Input, Flatten
from tensorflow.keras.constraints import max_norm
from tensorflow.keras import backend as K

from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.losses import BinaryCrossentropy



def EEGNet(nb_classes, Chans=64, Samples=128,
           dropoutRate=0.5, kernLength=128, F1=256,
           D=2, F2=128, norm_rate=0.25, dropoutType='SpatialDropout2D'):
    
    if dropoutType == 'SpatialDropout2D':
        dropoutType = SpatialDropout2D
    elif dropoutType == 'Dropout':
        dropoutType = Dropout
    else:
        raise ValueError('dropoutType must be one of SpatialDropout2D '
                         'or Dropout, passed as a string.')
    
    input1   = Input(shape = (Chans, Samples, 1))

    ##################################################################
    block1       = Conv2D(F1, (1, kernLength), padding = 'same',
                                   input_shape = (Chans, Samples, 1),
                                   use_bias = False)(input1)
    block1       = BatchNormalization()(block1)
    block1       = DepthwiseConv2D((Chans, 1), use_bias = False, 
                                   depth_multiplier = D,
                                   depthwise_constraint = max_norm(1.))(block1)
    block1       = BatchNormalization()(block1)
    block1       = Activation('elu')(block1)
    block1       = AveragePooling2D((1, 4))(block1)
    block1       = dropoutType(dropoutRate)(block1)
    
    block2       = SeparableConv2D(F2, (1, 16),
                                   use_bias = False, padding = 'same')(block1)
    block2       = BatchNormalization()(block2)
    block2       = Activation('elu')(block2)
    block2       = AveragePooling2D((1, 8))(block2)
    block2       = dropoutType(dropoutRate)(block2)
        
    flatten      = Flatten(name = 'flatten')(block2)
    
    dense        = Dense(256,  
                         kernel_constraint = max_norm(norm_rate))(flatten)
    dense       = Dropout(dropoutRate)(dense)
    
    dense        = Dense(nb_classes,
                         kernel_constraint = max_norm(norm_rate))(dense)
    softmax      = Activation('softmax', name = 'softmax')(dense)
    
    return Model(inputs=input1, outputs=softmax)

model_3 = EEGNet(2, Chans=data_array.shape[1],Samples=data_array.shape[2] )
model_3.summary()

c:\Users\Atharva\Anaconda3\envs\py_tf\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 21, 30951, 1)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 21, 30951, 256) │        32,768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_9           │ (None, 21, 30951, 256) │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ depthwise_conv2d_3              │ (None, 1, 30951, 512)  │        10,752 │
│ (DepthwiseConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_10          │ (None, 1, 30951, 512)  │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_6 (Activation)       │ (None, 1, 30951, 512)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_6             │ (None, 1, 7737, 512)   │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout2d_4             │ (None, 1, 7737, 512)   │             0 │
│ (SpatialDropout2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ separable_conv2d_3              │ (None, 1, 7737, 128)   │        73,728 │
│ (SeparableConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_11          │ (None, 1, 7737, 128)   │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_7 (Activation)       │ (None, 1, 7737, 128)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_7             │ (None, 1, 967, 128)    │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout2d_5             │ (None, 1, 967, 128)    │             0 │
│ (SpatialDropout2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 123776)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │    31,686,912 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 2)              │           514 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ softmax (Activation)            │ (None, 2)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 31,808,258 (121.34 MB)

 Trainable params: 31,806,466 (121.33 MB)

 Non-trainable params: 1,792 (7.00 KB)

In [32]:
# Compile model
model_3.compile(optimizer='adam', loss=BinaryCrossentropy(),
                metrics=['accuracy'])

# Train model on current fold's train and validation data

model_3.fit(train_features, train_labels, epochs=5,batch_size=1, validation_split=0.2)

# Evaluate model on test data for this fold
test_loss, test_acc = model_3.evaluate(test_features, test_labels)

Epoch 1/5
45/45 ━━━━━━━━━━━━━━━━━━━━ 127s 3s/step - accuracy: 0.4157 - loss: 1.0428 - val_accuracy: 0.7500 - val_loss: 0.6824
Epoch 2/5
45/45 ━━━━━━━━━━━━━━━━━━━━ 123s 3s/step - accuracy: 0.8583 - loss: 0.6412 - val_accuracy: 0.2500 - val_loss: 0.7350
Epoch 3/5
45/45 ━━━━━━━━━━━━━━━━━━━━ 133s 3s/step - accuracy: 0.4376 - loss: 0.8678 - val_accuracy: 0.7500 - val_loss: 0.6451
Epoch 4/5
45/45 ━━━━━━━━━━━━━━━━━━━━ 123s 3s/step - accuracy: 0.9185 - loss: 0.3591 - val_accuracy: 0.6667 - val_loss: 0.6485
Epoch 5/5
45/45 ━━━━━━━━━━━━━━━━━━━━ 123s 3s/step - accuracy: 0.8005 - loss: 0.3855 - val_accuracy: 0.3333 - val_loss: 0.7251
1/1 ━━━━━━━━━━━━━━━━━━━━ 94s 94s/step - accuracy: 0.1333 - loss: 0.8558


### Model 4


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Permute, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import SeparableConv2D, DepthwiseConv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import SpatialDropout2D
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.layers import Input, Flatten
from tensorflow.keras.constraints import max_norm
from tensorflow.keras import backend as K

from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.losses import BinaryCrossentropy



def EEGNet(nb_classes, Chans=64, Samples=128,
           dropoutRate=0.5, kernLength=128, F1=256,
           D=2, F2=128, norm_rate=0.25, dropoutType='SpatialDropout2D'):
    
    if dropoutType == 'SpatialDropout2D':
        dropoutType = SpatialDropout2D
    elif dropoutType == 'Dropout':
        dropoutType = Dropout
    else:
        raise ValueError('dropoutType must be one of SpatialDropout2D '
                         'or Dropout, passed as a string.')
    
    input1   = Input(shape = (Chans, Samples, 1))

    ##################################################################
    block1       = Conv2D(F1, (1, kernLength), padding = 'same',
                                   input_shape = (Chans, Samples, 1),
                                   use_bias = False)(input1)
    block1       = BatchNormalization()(block1)
    block1       = DepthwiseConv2D((Chans, 1), use_bias = False, 
                                   depth_multiplier = D,
                                   depthwise_constraint = max_norm(1.))(block1)
    block1       = BatchNormalization()(block1)
    block1       = Activation('elu')(block1)
    block1       = AveragePooling2D((1, 4))(block1)
    block1       = dropoutType(dropoutRate)(block1)
    
    block2       = SeparableConv2D(F2, (1, 16),
                                   use_bias = False, padding = 'same')(block1)
    block2       = BatchNormalization()(block2)
    block2       = Activation('elu')(block2)
    block2       = AveragePooling2D((1, 8))(block2)
    block2       = dropoutType(dropoutRate)(block2)
        
    flatten      = Flatten(name = 'flatten')(block2)
    
    dense        = Dense(128,  
                         kernel_constraint = max_norm(norm_rate))(flatten)
    dense       = Dropout(dropoutRate)(dense)
    
    dense        = Dense(64,  
                         kernel_constraint = max_norm(norm_rate))(dense)
    dense       = Dropout(dropoutRate)(dense)
    dense        = Dense(nb_classes, name = 'dense', 
                         kernel_constraint = max_norm(norm_rate))(dense)
    softmax      = Activation('softmax', name = 'softmax')(dense)
    
    return Model(inputs=input1, outputs=softmax)

model_4 = EEGNet(2, Chans=data_array.shape[1],Samples=data_array.shape[2] )
model_4.summary()

Model: "functional_16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_16 (InputLayer)     │ (None, 21, 30951, 1)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_44 (Conv2D)              │ (None, 21, 30951, 256) │        32,768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_55          │ (None, 21, 30951, 256) │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ depthwise_conv2d_9              │ (None, 1, 30951, 512)  │        10,752 │
│ (DepthwiseConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_56          │ (None, 1, 30951, 512)  │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_53 (Activation)      │ (None, 1, 30951, 512)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_18            │ (None, 1, 7737, 512)   │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_46 (Dropout)            │ (None, 1, 7737, 512)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ separable_conv2d_9              │ (None, 1, 7737, 128)   │        73,728 │
│ (SeparableConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_57          │ (None, 1, 7737, 128)   │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_54 (Activation)      │ (None, 1, 7737, 128)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_19            │ (None, 1, 967, 128)    │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_47 (Dropout)            │ (None, 1, 967, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 123776)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 256)            │    31,686,912 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2)              │           258 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ softmax (Activation)            │ (None, 2)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 31,840,898 (121.46 MB)

 Trainable params: 31,839,106 (121.46 MB)

 Non-trainable params: 1,792 (7.00 KB)

In [ ]:
# Compile model
model_4.compile(optimizer='adam', loss=BinaryCrossentropy(),
                metrics=['accuracy'])

# Train model on current fold's train and validation data

model_4.fit(train_features, train_labels, epochs=2,batch_size=1, validation_split=0.2)

# Evaluate model on test data for this fold
test_loss, test_acc = model_4.evaluate(test_features, test_labels)

Epoch 1/3
45/45 ━━━━━━━━━━━━━━━━━━━━ 209s 3s/step - accuracy: 0.4618 - loss: 0.8337 - val_accuracy: 0.4167 - val_loss: 0.6950
Epoch 2/3
45/45 ━━━━━━━━━━━━━━━━━━━━ 124s 3s/step - accuracy: 0.7380 - loss: 0.4727 - val_accuracy: 0.7500 - val_loss: 0.6811
Epoch 3/3
45/45 ━━━━━━━━━━━━━━━━━━━━ 123s 3s/step - accuracy: 0.8272 - loss: 0.4244 - val_accuracy: 0.3333 - val_loss: 0.7341
1/1 ━━━━━━━━━━━━━━━━━━━━ 76s 76s/step - accuracy: 0.5333 - loss: 0.6892
